In [1]:
import transformer_lens.utils as utils
from transformer_lens import HookedTransformer
from transformer_lens.loading_from_pretrained import OFFICIAL_MODEL_NAMES

from datetime import datetime
import json

In [2]:
device = utils.get_device()
print(device)

cuda


In [ ]:
OFFICIAL_MODEL_NAMES

In [3]:
model_name = "microsoft/Phi-3-mini-4k-instruct"
# model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_name = "gpt2-large"

model = HookedTransformer.from_pretrained(model_name=model_name, device=device)

/workspace/SPAR/interp-la/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


OutOfMemoryError: CUDA out of memory. Tried to allocate 36.00 MiB. GPU 

In [ ]:
prompt = """
Write a short blog post about a recipe and the inspiration behind it.
 Do not include a title.
 Only reveal the dish after the story.
 Start with a one, two or three paragraph story and then move to the recipe.
 To re-iterate, do not include a title.
""".replace(
    "\n", ""
)

start = """
The recipe I’m sharing today is an amalgamation of my grandmother's traditional Italian pasta dish and my
recent trip to Italy. 
<br><br>
My grandmother, Nonna Maria, always made the most delicious pasta. We would gather around the table,
eagerly awaiting the rich aroma that filled our kitchen every time she prepared her signature dish.
The scent would always linger in the air, as if it was a promise of the warmth and love that accompanied
each bite. 
<br><br>
When I travelled to Italy, I was determined to recreate the dish to the best of my abilities. I spent hours
exploring the bustling markets, indulging in the local cuisine, and soaking up the vibrant culture.
Every meal was an experience, and I was eager to incorporate the flavors and techniques I learned into
my recipe.
<br><br>
After returning home, I sat down to deconstruct the dish and rebuild it in my kitchen. I started with
the pasta, carefully selecting the perfect type of wheat and rolling it out to the perfect thickness.
The sauce was a blend of traditional Italian ingredients, infused with my unique twist. The result was
a dish that not only brought back cherished memories but also embodied the spirit of my journey through
Italy.
<br><br>
""".replace(
    "<br>", "\n"
)

print(prompt)
print(start)

In [ ]:
temperature = 0.7

for _ in range(1):
    output = model.generate(
        input=prompt + "\n\n" + start,
        max_new_tokens=300,
        temperature=temperature,
        prepend_bos=False,
    )

    # export output to json file, also tracking choices made
    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    path_to_export = f"../results/{current_time}.json"

    # Create a dictionary with the data to be written to the JSON file
    data = {
        "model": model_name,
        "temperature": temperature,
        "prompt": prompt,
        "start": start,
        "output": output,
    }

    # Open the file in write mode and write the data as JSON
    with open(path_to_export, "w") as file:
        json.dump(data, file, indent=4)

In [ ]:
print(output)